In [8]:
import requests
from tqdm.notebook import tqdm
import json

In [9]:
base_url = "http://0.0.0.0:5000"

api_endpoints = [
    "http://0.0.0.0:5000" + "/api/inspect/describe_openaire/",
    "http://0.0.0.0:5000" + "/api/inspect/describe_opencitation/",
    "http://0.0.0.0:5000" + "/api/inspect/describe_wikidata/",
]

In [10]:
NB_DOIS = 2000

%run get_dois_dryad.ipynb
filename = "dryad_" + str(NB_DOIS) + ".json"

# %run get_dois_pangaea.ipynb
# filename = "pangaea_" + str(NB_DOIS) + ".json"

# %run get_dois_zenodo.ipynb
# filename = "zenodo_" + str(NB_DOIS) + ".json"

%store -r dois_list

REST request to dryad...
Parsing result...
https://datadryad.org/api/v2/datasets?page=2&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=3&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=4&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=5&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=6&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=7&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=8&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=9&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=10&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=11&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=12&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?pag

In [11]:
import random
#sample_tool = random.choice(index.keys())

random.seed(10)

print(len(dois_list))

#samples = random.sample(list(index.items()), 1000)
samples_dois_list = random.sample(dois_list, NB_DOIS)

splitedSize = 100
samples_chunks = [samples_dois_list[x:x+splitedSize] for x in range(0, len(samples_dois_list), splitedSize)]

3100


In [12]:
print(len(samples_dois_list))

2000


In [13]:
results = []
for url in tqdm(samples_dois_list):
    response = requests.get("http://0.0.0.0:5000/api/inspect/get_rdf_metadata/" + url)
    graph = json.dumps(response.json(), ensure_ascii=False)
    for endpoint in api_endpoints:
        response = requests.post(endpoint, json={"json-ld": graph, "url": url})
        result = {
            "url": url,
            "endpoint": endpoint.split("/")[-2],
            "triples_before": response.json()["triples_before"],
            "triples_after": response.json()["triples_after"],
            "improved": response.json()["triples_before"] < response.json()["triples_after"],
        }
        results.append(result)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [14]:
improved = 0
not_improved = 0

for res in results:
    if not res["improved"]:
        not_improved += 1
    else:
        print("ENDPOINT: " + res["endpoint"])
        print("URL: " + res["url"])
        improved += 1
        
print("Improved: " + str(improved))
print("Not improved: " + str(not_improved))

ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.f2n6f
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.h79q4
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.s00vr
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.451n7
ENDPOINT: describe_openaire
URL: https://doi.org/10.5061/dryad.p1b90
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.fs756
ENDPOINT: describe_openaire
URL: https://doi.org/10.5061/dryad.5n2bs6r
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.sm107
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.r6b87
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.4rd14
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.g60r3
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.s58qh
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5061/dryad.ch110
ENDPOINT: describe_opencitation
URL: https://doi.org/10.5

In [15]:
json_object = json.dumps(results)
with open("results/" + filename, "w+") as outfile:
    outfile.write(json_object)